In [2]:
import os
print(os.getcwd())
os.chdir('/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1')
print(os.getcwd())

/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1/tests
/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1


In [11]:
import pandas as pd
data=pd.read_csv("data/dev.tsv", sep='\t')
data

,sentence1,sentence2,label,relations
0,marriage is pase.,Not everyone believes in marriage anymore.,support,(marriage; capable of; deceiving)(deceiving; c...
1,marriage is the best for a family unit.,Marriage is a predictor of health and happiness.,support,(marriage; created by; love)(love; causes; hea...
2,Entrapment shouldn't be legalized since it put...,Entrapment is really a trick,support,(entrapment; is a; trick)(trick; is a; false s...
3,Marriage is the backbone of society.,Marriage forms strong families.,support,(marriage; used for; couples)(couples; capable...
4,Entrapment causes police to abuse citizens and...,Entrapment causes harm to citizens,support,(entrapment; capable of; being abused)(being a...
...,...,...,...,...
393,Human cloning is dangerous for society.,We do not know enough about human cloning to t...,counter,(human cloning; not has context; not know enou...
394,People should be allowed to believe fairy tale...,Some fairy tales are based on facts with a col...,support,(believable; has context; by people)(believabl...
395,Cities are too crowded and crime ridden.,Crimes occur in the country and suburbs.,counter,(crimes; part of; crime ridden)(crimes; capabl...
396,In order to not be linked to doing stuff by so...,Atheism provides a good excuse for not being t...,support,(belief systems; has context; religions)(relig...


In [12]:
import pandas as pd
import re

# 提取实体和关系
entity_set = set()
relationship_list = []

def extract_relations(row):
    relations = row['relations']
    sentence1 = row['sentence1']
    sentence2 = row['sentence2']
    label = row['label']
    
    # 匹配关系三元组，形式为：(entity1; relation; entity2)
    triplets = re.findall(r'\(([^;]+); ([^;]+); ([^)]+)\)', relations)
    for triplet in triplets:
        source, relation, target = triplet
        source = source.strip()
        target = target.strip()
        relation = relation.strip()
        
        # 添加实体
        entity_set.add((source, sentence1, sentence2, label))
        entity_set.add((target, sentence1, sentence2, label))
        
        # 记录关系，并将相关的上下文信息存储
        relationship_list.append((source, target, relation, sentence1, sentence2, label))

# 遍历数据集提取实体和关系
data.apply(extract_relations, axis=1)

# 创建实体表，赋予唯一ID，并保存sentence1, sentence2, label
entity_list = list(entity_set)
entity_df = pd.DataFrame(entity_list, columns=['entity_name', 'sentence1', 'sentence2', 'label'])
entity_df['entity_id'] = range(1, len(entity_list) + 1)

# 创建关系表，将实体映射为id，并保存sentence1, sentence2, label
entity_id_map = {name: idx for idx, (name, _, _, _) in enumerate(entity_list, 1)}

relationship_df = pd.DataFrame(relationship_list, columns=['source_entity', 'target_entity', 'relationship_type', 'sentence1', 'sentence2', 'label'])
relationship_df['source_entity_id'] = relationship_df['source_entity'].map(entity_id_map)
relationship_df['target_entity_id'] = relationship_df['target_entity'].map(entity_id_map)
relationship_df['relationship_id'] = range(1, len(relationship_df) + 1)

# 最终的关系表
relationship_df = relationship_df[['relationship_id', 'source_entity_id', 'source_entity', 'target_entity_id', 'target_entity', 'relationship_type', 'sentence1', 'sentence2', 'label']]
entity_df


,entity_name,sentence1,sentence2,label,entity_id
0,urbanization,Urbanization is terrible for society.,Urbanization increases jobs.,counter,1
1,austerity,austerity regimes exploit the poor.,Austerity rips the band aid faster.,counter,2
2,embryonic stem cell,Embryonic stem cell research is very expensive...,Embryonic stem cell research helps fight disease.,support,3
3,less benefits,lack of government spending can be detrimental...,Unemployment can be exaserbated by reduced gov...,support,4
4,be closed down,Guantanamo bay should not be closed down.,Guantanamo bay is notorious for its use of tor...,counter,5
...,...,...,...,...,...
2149,treated fairly,Guantanamo Bay Detention camp should stay oper...,All humans deserve to be treated fairly and cr...,counter,2150
2150,number of people,Atheism restricts religious freedom,The increased number of people becoming Atheis...,counter,2151
2151,confusion,Human cloning should be allowed.,Human cloning is terrifying.,counter,2152
2152,harmful to babies,Embryonic stem cell research is important.,Embryonic stem cell research is harmful to bab...,counter,2153


In [13]:
relationship_df

,relationship_id,source_entity_id,source_entity,target_entity_id,target_entity,relationship_type,sentence1,sentence2,label
0,1,2092,marriage,1547,deceiving,capable of,marriage is pase.,Not everyone believes in marriage anymore.,support
1,2,1547,deceiving,584,pase,created by,marriage is pase.,Not everyone believes in marriage anymore.,support
2,3,584,pase,1769,everyone,used for,marriage is pase.,Not everyone believes in marriage anymore.,support
3,4,1769,everyone,1202,believes,capable of,marriage is pase.,Not everyone believes in marriage anymore.,support
4,5,2092,marriage,2099,love,created by,marriage is the best for a family unit.,Marriage is a predictor of health and happiness.,support
...,...,...,...,...,...,...,...,...,...
1788,1789,2059,society,1334,hard working people,part of,People who created budget deficits should be t...,Rich people are not the only ones who need to ...,counter
1789,1790,1452,taxes,1402,rich people,created by,People who created budget deficits should be t...,Rich people are not the only ones who need to ...,counter
1790,1791,1485,government spending,1471,benefits,used for,People who created budget deficits should be t...,Rich people are not the only ones who need to ...,counter
1791,1792,1452,taxes,1334,hard working people,created by,People who created budget deficits should be t...,Rich people are not the only ones who need to ...,counter


In [8]:
import ollama
import pandas as pd
from tqdm import tqdm

# 初始化全局变量来记录token数量
token_num = 0

# 定义一个函数，使用LLM生成实体的类型和描述，要求严格的格式输出
def generate_entity_type_and_description(entity, sentence1, sentence2, label):
    global token_num  # 声明使用全局变量

    prompt = f"""
    Context:
    - Sentence 1: {sentence1}
    - Sentence 2: {sentence2}
    - Label: {label}

    Now, please provide the following information for the entity: '{entity}'.
    1. Type: Describe what kind of entity it is (e.g., concept, person, organization). Output format: "Type: <entity type>"
    2. Description: Provide a brief description of the entity. Output format: "Description: <entity description>"

    Please follow this format strictly:
    Type: <entity type>
    Description: <entity description>

    Don't say any other language or words!
    """
    
    # 调用ollama模型
    response = ollama.chat(model='gemma2:2b', messages=[
        {
            'role': 'user',
            'content': prompt,
        },
    ])
    
    # 处理模型的返回结果，提取Type和Description
    result = response['message']['content'].strip()
    type_result = ""
    description_result = ""

    # 使用正则表达式或简单字符串解析提取type和description
    if "Type:" in result and "Description:" in result:
        type_result = result.split("Type:")[1].split("Description:")[0].strip()
        description_result = result.split("Description:")[1].strip()

    # 从响应中提取token使用信息（假设响应中包含'token_usage'键）
    if 'token_usage' in response:
        token_num += response['token_usage'].get('total', 0)  # 增加token数量
    
    return type_result, description_result

# 使用tqdm进度条包装apply方法
tqdm.pandas()  # 激活tqdm与pandas结合

# 在entity_df上分别为每个实体生成type和description，带进度条显示
entity_df[['type', 'description']] = entity_df.progress_apply(
    lambda row: pd.Series(generate_entity_type_and_description(row['entity_name'], row['sentence1'], row['sentence2'], row['label'])),
    axis=1
)

# 打印生成的entity_df和消耗的token数量
print(entity_df)
print(f"Total tokens used: {token_num}")

# 如果需要，可以保存结果
entity_df.to_csv('tests/export/entity_with_type_and_description.csv', index=False)


100%|██████████| 273/273 [06:09<00:00,  1.35s/it]

                  entity_name  \
0                    marriage   
1            breaking the law   
2    inclusive of nonbinaries   
3                   pollution   
4             catch criminals   
..                        ...   
268                   useless   
269                  cultures   
270          tricks criminals   
271        catching criminals   
272                   couples   

                                             sentence1  \
0                          marriage has many benefits.   
1    Entrapment shouldn't be legal, because it crea...   
2    The idea of marriage can be changed, without a...   
3                urbanization is terrible for society.   
4    law enforcement can use entrapment to catch cr...   
..                                                 ...   
268                      marriage is useless nowadays.   
269                              Marriage is a burden.   
270  Entrapment isn't fair to the person being trap...   
271                    

In [9]:
entity_df

,entity_name,sentence1,sentence2,label,entity_id,type,description
0,marriage,marriage has many benefits.,Marriage offers tax benefits.,support,1,Concept,A socially recognized union between two people...
1,breaking the law,"Entrapment shouldn't be legal, because it crea...",Entrapment tricks criminals to behave a certai...,support,2,Concept,"The act of violating a legal code, often invol..."
2,inclusive of nonbinaries,"The idea of marriage can be changed, without a...",Marriage needs to evolve.,support,3,Concept,A concept that acknowledges and embraces the f...
3,pollution,urbanization is terrible for society.,urbanization makes society worse off.,support,4,concept,A general term for the release of harmful subs...
4,catch criminals,law enforcement can use entrapment to catch cr...,Sometimes you can't have sufficient evidence t...,support,5,Concept,The act of apprehending criminals.
...,...,...,...,...,...,...,...
268,useless,marriage is useless nowadays.,Most people in society get married.,counter,269,Concept,A quality or characteristic indicating somethi...
269,cultures,Marriage is a burden.,Marriage as an institution has existed and wil...,counter,270,Concept,"Cultures represent diverse societies, practice..."
270,tricks criminals,Entrapment isn't fair to the person being trap...,Entrapment tricks criminals to behave a certai...,support,271,Concept,A method of manipulating someone into engaging...
271,catching criminals,entrapment is efficient.,Entrapment catches criminals.,support,272,concept,The act of apprehending or arresting criminals.


In [3]:
import pandas as pd
data_train=pd.read_csv("data/train.tsv", sep='\t')
data_train

,sentence1,sentence2,label,relations
0,Cannabis should be legal.,It's not a bad thing to make marijuana more av...,support,(cannabis; synonym of; marijuana)(legal; cause...
1,Women should not be in combat.,Women and men have the same rights.,counter,(women and men; is a; citizens)(citizens; caus...
2,People will use marijuana independent of its l...,People use marijuana everywhere now.,support,(marijuana; receives action; popular)(popular;...
3,women should not partake in war,the armed forces are more open to recruiting w...,counter,(armed forces; desires; nurses and helpers)(nu...
4,Marijuana should not be legalized.,Marijuana is dangerous for society.,support,(marijuana; is a; recreational drug)(recreatio...
...,...,...,...,...
2363,telemarketing should be dictated by the market.,Telemarketing offers no value to society.,counter,(telemarketing; not capable of; value to socie...
2364,Acting takes children away from their educatio...,Some children enjoy acting.,counter,(some children; capable of; acting)(acting; ha...
2365,Some people's skills are better suited to area...,The military has a need for a great many skills.,counter,(military; has property; main purpose)(main pu...
2366,School prayer allows student to do what they a...,School prayer should be discouraged as it teac...,counter,(parents; has property; encourage children's b...


In [13]:
import pandas as pd
import re

# 提取实体和关系
entity_dict = {}
relationship_list = []

def normalize_entity_name(entity_name):
    # 统一实体名称的格式，移除空格、符号，转换为小写等
    return entity_name.strip().lower()

def extract_relations(row):
    relations = row['relations']
    sentence1 = row['sentence1']
    sentence2 = row['sentence2']
    label = row['label']
    
    # 匹配关系三元组，形式为：(entity1; relation; entity2)
    triplets = re.findall(r'\(([^;]+); ([^;]+); ([^)]+)\)', relations)
    for triplet in triplets:
        source, relation, target = triplet
        source = normalize_entity_name(source)
        target = normalize_entity_name(target)
        relation = relation.strip()
        
        # 添加实体，避免重复
        if source not in entity_dict:
            entity_dict[source] = (source, sentence1, sentence2, label)
        if target not in entity_dict:
            entity_dict[target] = (target, sentence1, sentence2, label)
        
        # 记录关系，并将相关的上下文信息存储
        relationship_list.append((source, target, relation, sentence1, sentence2, label))

# 遍历数据集提取实体和关系
data_train.apply(extract_relations, axis=1)

# 创建实体表，赋予唯一ID，并保存sentence1, sentence2, label
entity_list = list(entity_dict.values())
entity_df = pd.DataFrame(entity_list, columns=['entity_name', 'sentence1', 'sentence2', 'label'])
entity_df['entity_id'] = range(1, len(entity_list) + 1)

# 创建实体名称到实体ID的映射
entity_id_map = {name: idx for idx, (name, _, _, _) in enumerate(entity_list, 1)}

# 创建关系表，将实体映射为id，并保存sentence1, sentence2, label
relationship_df = pd.DataFrame(relationship_list, columns=['source_entity', 'target_entity', 'relationship_type', 'sentence1', 'sentence2', 'label'])

# 更新关系表中的实体ID映射
relationship_df['source_entity_id'] = relationship_df['source_entity'].map(entity_id_map)
relationship_df['target_entity_id'] = relationship_df['target_entity'].map(entity_id_map)
relationship_df['relationship_id'] = range(1, len(relationship_df) + 1)

# 最终的关系表
relationship_df = relationship_df[['relationship_id', 'source_entity_id', 'source_entity', 'target_entity_id', 'target_entity', 'relationship_type', 'sentence1', 'sentence2', 'label']]

# 打印或保存结果
entity_df

,entity_name,sentence1,sentence2,label,entity_id
0,cannabis,Cannabis should be legal.,It's not a bad thing to make marijuana more av...,support,1
1,marijuana,Cannabis should be legal.,It's not a bad thing to make marijuana more av...,support,2
2,legal,Cannabis should be legal.,It's not a bad thing to make marijuana more av...,support,3
3,more available,Cannabis should be legal.,It's not a bad thing to make marijuana more av...,support,4
4,good thing,Cannabis should be legal.,It's not a bad thing to make marijuana more av...,support,5
...,...,...,...,...,...
6326,provides diverse candidates,A multi-party system decreases votes of indivi...,A multi-party system ensures that only a good ...,counter,6327
6327,voter options,A multi-party system decreases votes of indivi...,A multi-party system ensures that only a good ...,counter,6328
6328,good candidate,A multi-party system decreases votes of indivi...,A multi-party system ensures that only a good ...,counter,6329
6329,bad candidate,A multi-party system decreases votes of indivi...,A multi-party system ensures that only a good ...,counter,6330


In [14]:
relationship_df

,relationship_id,source_entity_id,source_entity,target_entity_id,target_entity,relationship_type,sentence1,sentence2,label
0,1,1,cannabis,2,marijuana,synonym of,Cannabis should be legal.,It's not a bad thing to make marijuana more av...,support
1,2,3,legal,4,more available,causes,Cannabis should be legal.,It's not a bad thing to make marijuana more av...,support
2,3,2,marijuana,5,good thing,capable of,Cannabis should be legal.,It's not a bad thing to make marijuana more av...,support
3,4,5,good thing,3,legal,desires,Cannabis should be legal.,It's not a bad thing to make marijuana more av...,support
4,5,6,women and men,7,citizens,is a,Women should not be in combat.,Women and men have the same rights.,counter
...,...,...,...,...,...,...,...,...,...
9947,9948,6327,provides diverse candidates,6328,voter options,causes,A multi-party system decreases votes of indivi...,A multi-party system ensures that only a good ...,counter
9948,9949,6328,voter options,6329,good candidate,capable of,A multi-party system decreases votes of indivi...,A multi-party system ensures that only a good ...,counter
9949,9950,6328,voter options,6330,bad candidate,capable of,A multi-party system decreases votes of indivi...,A multi-party system ensures that only a good ...,counter
9950,9951,6329,good candidate,6331,large number votes,receives action,A multi-party system decreases votes of indivi...,A multi-party system ensures that only a good ...,counter


In [15]:
from src.extractors.graph_extractor import GraphExtractor
import pandas as pd

# 初始化 GraphExtractor
graph_extractor = GraphExtractor()

# 生成图并执行社区检测
G = graph_extractor.build_graph(entity_df, relationship_df)
community_mapping, sub_community_mapping = graph_extractor.perform_community_detection()


Clustering sub-communities: 100%|██████████| 64/64 [00:00<00:00, 402.61it/s]


In [17]:
# 计算社区的数量
unique_communities = set(community_mapping.values())

# 计算子社区的唯一组合 (community_id, sub_community_id)
unique_sub_communities = set((community_mapping[node], sub_community_mapping[node]) for node in sub_community_mapping)

# 计算社区和子社区数量
num_communities = len(unique_communities)
num_sub_communities = len(unique_sub_communities)

# 打印结果
print(f"Total number of communities: {num_communities}")
print(f"Total number of unique sub-communities: {num_sub_communities}")


Total number of communities: 64
Total number of unique sub-communities: 586
